
# SQLAlchemy

SQLAlchemy의 연결 풀에 대한 기본 개념과 실전에서 연결 풀링과 관하여 알면 좋을 여러 이슈



# 연결 풀링 개념

연결 풀링은 차후에 발생할 데이터베이스 요청에 대비하여 데이터베이스 연결을 캐싱하는 기법 
빈번한 데이터베이스 요청이 여러 사용자에 의해 발생할 때, 매번 연결을 생성하고 닫는 과정을 반복하면 이에 대한 비용이 크기 때문에 
이 기법을 사용하여 연결 생성 과정을 줄일 수 있다. 
짧은 요청이 빈번하게 발생하는 웹 서비스와 같은 형태가 연결 풀과 궁합이 잘 맞다.



# SQLAlchemy의 기본 풀: 큐 풀(QueuePool)

SQLAlchemy 역시 연결 풀을 기본적으로 채택하고 있는데, 그중 기본으로 제공하는 것은 큐 풀(QueuePool) 
큐 풀은 설정된 pool_size와 max_overflow를 바탕으로 복수의 연결 풀을 구성해서 운용
SQLite를 제외한 모든 데이터베이스에서 기본값으로 이용 
큐 풀의 관리 방법 알아보기



# 큐 풀의 생애주기

1) 큐 풀이 처음부터 연결을 미리 만드는 것은 아니다. 일단 0개로 시작.
2) 요청이 들어올 때, 큐 풀에 유효 연결이 없으면 하나 생성.
3) 설정된 pool_size까지는 더 연결이 필요하지 않은 상황이라도 연결을 종료하지 않음.
4) 요청이 들어올 때, pool_size까지 다 찼다 할지라도 유효 연결이 없으면 초과하여 하나 생성.
5) 4번 이후부터는 overflow 상황 > 큐 풀은 적극적으로 overflow를 방지하기 위해 새로 들어오는 연결을 종료 > pool_size에 총연결 수를 맞춤.
6) QueuePool이 관리하는 연결이 pool_size + max_overflow까지 다 찬 상황에서 요청이 들어오면, 일단 기다리게 함.
7) 기본값으로는 30초를 기다린다. 30초를 기다려도 반환되는 연결이 없다면 TimeoutError 예외를 발생시킴.



# 적절한 큐 풀 설정값

서비스가 작을 때는 기본값이면 충분하지만, 서비스 사용량이 많아지고 규모 문제가 발생하게 된다면 설정을 현재 상황에 맞춰 바꿔주는 게 좋다. 
보통 QueuePool 관련 위 언급한 2가지 값(pool_size, max_overflow)을 바꿔주는 게 좋은데 기본값은 (5, 10)이다.

pool_size:      현재 구성에서 연결 생성 부담을 최소화할 수 있는 가장 작은 값이 되어야 함.
                과하게 설정되어있으면 데이터베이스 입장에서 너무 많은 연결을 점유하고 있으니 비효율적이다.
                그렇다고, 너무 적게 설정한다면 오버플로가 자주 발생하여 풀링으로 얻을 수 있는 효율을 누리지 못한다. 
                즉, 파이썬 측에서 비효율적이다.
                
max_overflow:   현재 구성에서 데이터베이스, 웹 인스턴스가 물리적으로 버틸 수 있는 최댓값이 되어야 함.
                max_overflow가 데이터베이스나 웹 인스턴스의 한계치보다 너무 빡빡하게 잡혀있으면 조금만 사용자 유입이 늘어도 
                TimeoutError를 쉽게 만나거나 서비스 속도 저하를 자주 경험하게 됨.
                그렇다고 무한으로 두면 사용량 폭증시 이해할 수 없는 에러가 다량 발생.(데이터베이스나 파이썬 앱, 혹은 둘 다 드러누움.)

결국 서비스마다 그만의 퍼포먼스와 장비 한계치가 있으니만큼 내부에서 스트레스 테스트를 통한 벤치마킹으로 적정 값을 뽑아내는 것을 추천.



# 큐 풀 관하여 자주 밟는 문제

>개발할 때는 문제가 없었는데, 상용 서버를 띄우면 수분 이내로 서버가 TimeoutError 예외를 발생하며 응답을 안 합니다.
    SQLAlchemy 쓰는 서비스를 만들어서, 개발 잘 하고 배포했는데 프로덕션에서 잠깐 잘 돌더니 TimeoutError를 내뱉으며 죽어버리는 경우가 많음. 
    이 에러 자체는 Session이 큐 풀에 연결을 받기 위해 기다리다가 못 참고 TimeoutError를 내는 것. 
    위의 생애주기 기준, 7번에 해당하는 상황. 
    큐 풀의 timeout 기본값은 30이니까 30초 동안 풀의 모든 연결이 점유된 상태에서 아무것도 받지 못한 상태가 된 것.

    위와 같은 경험이라면 서비스 사용량이 폭증하는 쪽보다는 십중팔구 기존에 점유한 Session에서 제대로 연결을 반환해주지 않아서 발생하는 문제. 
    특히 웹서비스라면 Flask 등에서 요청 시마다 Session이 연결을 불러다 써놓고 Pool에 돌려주는 일을 빼먹는 실수가 잦은데, 
    Flask를 쓰고 계신다면 Flask-SQLAlchemy 등을 쓰셔서 생애주기 관리 자체를 타 라이브러리에 위임하거나, 
    현재 구조상에서 요청이 끝나는 시점에 맞춰 session.close()를 적절히 호출해주면 해결됨. 
    (사실 Flask-SQLAlchemy가 해주는 것도 딱 이 정도의 수준)

> 어느 날 갑자기 연결이 왕창 늘어버렸어요.
    역시 웹서비스 개발하다보면 발생하는 이슈. 
    SQLAlchemy를 쓰면 Session 활용을 암시적으로 하게 될 때가 많다. 
    Session이 실제로 요청을 보내는 시점에서야 연결을 시도하기 때문에, 예상치 못한 기능 변경으로 연결 폭증을 겪는 것. 
    자주 있는 경우는 Flask의 생애주기중 before_request 구현에서 데이터베이스에 접근하는 것.

    본래 데이터베이스 연결이 필요한 엔드포인트에서만 접속이 발생하던 것이, 
    before_request에 붙으면서 모든 엔드포인트가 데이터베이스 연결을 하게 되면 사용량이 폭증하기 쉽다. 
    이처럼 전역적인 영역에서 DB 접근을 하는 시나리오를 최소화하는 정책으로 실수를 완화 가능.
    


# 결론

SQLAlchemy의 연결 풀의 동작 방식을 이해하면 상용 서비스를 운영할 때 발생하는 데이터베이스 부하 문제를 진단하고 해결하는 데 많은 도움이 된다. 
pool_size와 max_overflow의 적정값은 서비스에 따라, 인프라의 사양에 따라 다르므로 
이를 잘 파악하여 효율적으로 연결 풀이 운영될 수 있도록 세팅하는 것을 추천.

연결 풀을 관리하는 방법으로는 SQLAlchemy내의 기본 큐 풀을 쓰는 것 외에 Pgpool-II과 같은 미들웨어를 연결하는 방안도 있다. 
추후 이에 대해서도 다루어보기.

SQLAlchemy 0.7부터 SQLite 같은 파일 기반 데이터베이스에서는 기본적으로 NullPool을 채택. 
파일 기반 데이터베이스에는 네트워크 연결이 일어나지 않기 때문에, 연결 비용이 적기 때문.
NullPool은 이름에서 알 수 있듯이 연결 풀을 유지하지 않고 풀에 연결이 들어오는 즉시 폐기.

큐 풀의 pool_size를 0으로 하는 것과 같다고 착각할 수 있으나, 
큐 풀은 pool_size가 0일 때 pool_size가 무한대인 것으로 인식. 
따라서 풀을 만들지 않으려면 NullPool을 쓰는 것이 적절.



# engine.dispose()

python에서 DB연결 시, 주로 SQLAlchemy를 사용.
session을 설정 후 쿼리를 하고 나서 프로그램을 끝내기 전에는 session.close() 명령어를 사용해줘만 DB에서 커넥션을 잡고 있지 않음.
보통 for문 등 반복문에서 DB에서 가져온 리소스를 불러온 뒤 각 항목마다 업데이트를 하는데
for문 안에서 session.close()를 해주는 것이 라니라 모든 commit작업이 완료된 후에 close를 해준다.
또 프로그램 종료시에는 engine.dispose()를 통해 커넥션을 완전히 끊어준다. 
여기서 엔진은 engine_create로 정의한 커넥션을 의미한다.

그렇지 않은 경우 아래와 같은 비정상 종료로 로그를 남기며 쌓이고 결국 서버가 다운되기도 한다.

[Warning]Aborted connection 91179 to db: '*** user:'***host:'localhost'(Got an error reading communication packets)



# DB 취득 값 Dataframe 저장

DB에서 취득하고 싶은 내용을 작성한 쿼리를 read_sql()를 사용해 실행하면 결과값을 Dataframe 형태로 얻을 수 있다.
SQL Server에 아래와 같은 샘플 TEST라는 테이블을 만들어 값을 취득하는 방법


In [ ]:
import pyodbc
import pandas as pd

# 서버 주소(DB 접속ip)
server = "서버 주소"

# 접속 유저
user = "유저"

# 패스워드
pasword = "패스워드"

# 데이터베이스명
db = "데이터베이스명"

cnxn = "DRIVER={SQL Server};SERVER=" + server + ";uid=" + user + ";pwd=" + pasword + ";DATABASE=" + db
cursor = cnxn.cursor()

cnxn =  pymssql.connect(server , username, password,database,autocommit = False)
cursor = cnxn.cursor()

sql = 'SELECT * FROM TEST'
df = pd.read_sql(sql, cnxn)
print(df)

cursor.close()

[ 결과 ]

     col1    col2    col3
0    val1    val2    val3
1   data1   data2   data3
2  value1  value2  value3

------------------------------------------------------------
read_sql()로 쿼리를 실행해 결과 값을 Dataframe으로 저장
결과를 출력해보면 Dataframe의 컬럼명이 취득한 테이블 컬럼명이 된다
테이블 컬럼명과 Dataframe 컬럼명이 같기 때문에 접근하기가 편하다.

반복문을 사용해 한 줄씩 값을 취득해보자

In [ ]:
import pyodbc
import pandas as pd

# 서버 주소(DB 접속ip)
server = "서버 주소"

# 접속 유저
user = "유저"

# 패스워드
pasword = "패스워드"

# 데이터베이스명
db = "데이터베이스명"

cnxn = "DRIVER={SQL Server};SERVER=" + server + ";uid=" + user + ";pwd=" + pasword + ";DATABASE=" + db
cursor = cnxn.cursor()

cnxn =  pymssql.connect(server , username, password,database,autocommit = False)
cursor = cnxn.cursor()

sql = 'SELECT * FROM TEST'
df = pd.read_sql(sql, cnxn)

for index, row in df.iterrows():
    print("col1 값 : " + row.col1)
    print("col2 값 : " + row.col2)
    print("col3 값 : " + row.col3)

cursor.close()

[ 결과 ]

col1 값 : val1
col2 값 : val2
col3 값 : val3
col1 값 : data1
col2 값 : data2
col3 값 : data3
col1 값 : value1
col2 값 : value2
col3 값 : value3
    
-------------------------------------------------
for문으로 취득한 모든 행을 한 줄씩 처리한다
for문에서 선언한 row에는 dataframe에 저장된 레코드를 한 줄씩 대입
한 줄씩 대입된 row에서 row.컬럼이름을 지정해 해당 컬럼에 접근이 가능하다
취득한 값을 가공해야 하거나 데이터 건수가 많아 Dataframe에 저장해 사용하고 싶은 경우에는 
판다스 read_sql()를 사용해 직접 쿼리를 실행해 Dataframe 형태로 사용하는 것이 편리함